# Tutorial: Pump Unit Model with IAPWS Property Package



<img src="pump.svg" alt="drawing" width="200"/>



## Learning Outcomes

- Demonstrate use of the Pump unit model in IDAES
- Demonstrate different simulation options available


## Problem Statement

In this example, we will pump a stream of water at atmospheric pressure to a pressure of 201325 Pa using a simple pump unit model. We will be using the IAPWS property package for the water properties. It is assumed that the pump operates at steady state.

The inlet specifications are as follows:

* Flow Rate = 100 mol/s
* Mole fraction (H2O) = 1
* Pressure = 101325 Pa
* Temperature = 298.15 K

We will simulate 2 different cases, depending on the operating specifications by the user:

**Case 1**:In this case, we will specify the pump efficiency and the pressure increase variable on the pump unit model.

   * Pressure Increase = 100000 Pa
   * Pump Efficiency = 0.8
        
**Case 2**: In this case, we will specify the pump efficiency but will specify the pressure of the outlet stream directly. 
   * Outlet Pressure = 201325 Pa
   * Pump efficiency = 0.8

IDAES documentation reference for pump model: https://idaes-pse.readthedocs.io/en/stable/

## Setting up the problem in IDAES

In the following cell, we will be importing the necessary components from Pyomo and IDAES.

In [ ]:
# Import objects from pyomo package 
from pyomo.environ import ConcreteModel, SolverFactory, value, units

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False}) # dynamic or ss flowsheet needs to be specified here


# Import the IAPWS property package to create a properties block for the flowsheet
from idaes.generic_models.properties import iapws95
from idaes.generic_models.properties.helmholtz.helmholtz import (
    PhaseType
)

# Add properties parameter block to the flowsheet with specifications
m.fs.properties = iapws95.Iapws95ParameterBlock(default={"phase_presentation":PhaseType.L})

## Case 1: Fix pressure change and pump efficiency 


### Add Pump Unit Model

In [ ]:
# Import pump unit model from the model library
from idaes.generic_models.unit_models.pressure_changer import Pump

# Create an instance of the pump unit, attaching it to the flowsheet
# Specify that the property package to be used with the pump is the one we created earlier.
m.fs.pump_case_1 = Pump(default={"property_package": m.fs.properties})

In [ ]:
# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Call the degrees_of_freedom function, get intitial DOF
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

### Fix Inlet Stream Conditions

In [ ]:
# Fix the stream inlet conditions
m.fs.pump_case_1.inlet.flow_mol[0].fix(100) #mol/s

# Use the htpx method to obtain the molar enthalpy of inlet stream based on given conditions of temperature and pressure
m.fs.pump_case_1.inlet.enth_mol[0].fix(value(iapws95.htpx(T=298.15*units.K, P=101325*units.Pa))) #J/mol
m.fs.pump_case_1.inlet.pressure[0].fix(101325) #Pa

### Fix Pressure Change and Pump Efficiency

In [ ]:
# Fix pump conditions
m.fs.pump_case_1.deltaP.fix(100000)
m.fs.pump_case_1.efficiency_pump.fix(0.8)

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The final DOF is {0}".format(DOF_final))

### Initialization

In [ ]:
# Initialize the flowsheet, and set the logger level to INFO
m.fs.pump_case_1.initialize(outlvl=idaeslog.INFO)
 

### Solve Model

In [ ]:
# Solve the simulation using ipopt
# Note: If the degrees of freedom = 0, we have a square problem
opt = SolverFactory('ipopt')
solve_status = opt.solve(m, tee=True)

### View Results

In [ ]:
# Display a readable report
m.fs.pump_case_1.report()

## Case 2: Fix outlet pressure and pump efficiency


### Add Pump Unit Model

In [ ]:
# Create an instance of another pump unit, attaching it to the same flowsheet
# Specify that the property package to be used with the pump is the one we created earlier.
m.fs.pump_case_2 = Pump(default={"property_package": m.fs.properties})

# Call the degrees_of_freedom function, get intitial DOF
DOF_initial = degrees_of_freedom(m.fs.pump_case_2)
print("The initial DOF is {0}".format(DOF_initial))

### Fix Inlet Stream Conditions

In [ ]:
# Fix the stream inlet conditions
m.fs.pump_case_2.inlet.flow_mol[0].fix(100) # mol/s

# Use the htpx method to obtain the molar enthalpy of inlet stream based on given conditions of temperature and pressure
m.fs.pump_case_2.inlet.enth_mol[0].fix(value(iapws95.htpx(T=298.15*units.K, P=101325*units.Pa))) # J/mol
m.fs.pump_case_2.inlet.pressure[0].fix(101325) # Pa

### Fix Outlet Pressure & Pump Efficiency

In [ ]:
# Fix outlet stream conditions
m.fs.pump_case_2.outlet.pressure[0].fix(201325)

# Fix pump efficiency
m.fs.pump_case_2.efficiency_pump.fix(0.8)

In [ ]:
DOF_final = degrees_of_freedom(m.fs.pump_case_2)
print('The final degrees of freedom is: {0}'.format(DOF_final))

### Initialization

In [ ]:
# Initialize the flowsheet, and set the logger level to INFO
m.fs.pump_case_2.initialize(outlvl=idaeslog.INFO)


### Solve Model

In [ ]:
# Solve the simulation using ipopt
# Note: If the degrees of freedom = 0, we have a square problem
opt = SolverFactory('ipopt')
solve_status = opt.solve(m.fs.pump_case_2, tee=True)

### View Results

In [ ]:
# Display a readable report
m.fs.pump_case_2.report()